In [39]:
import requests
import pandas as pd

from dotenv import load_dotenv
import os
load_dotenv()

gameName = "BakutoPhinn"
tagLine = "NA1"
api_key = os.environ.get("riot_api")

In [30]:
def get_puuid(gameName, tagLine, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}"
    response = requests.get(link)
    return response.json()['puuid']

def get_name_and_tag(puuid, api_key):
    link = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
    response = requests.get(link)
    return [response.json()['gameName'], response.json()['tagLine']]

In [31]:
get_name_and_tag(get_puuid("crazy%20noche", "NA1", api_key), api_key)[0]

'Crazy Noche'